# Interactive Functions 

In [1]:
#! pip install plotnine

In [1]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [3]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False).drop('Unnamed: 0', axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [4]:
df.sample(5)

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
16553,HSIP,5932(083),Tuolumne County,2020-04-08,663300.00,-663300.00,0.00,Authorized,5932,10,HSIP_RURL,On Parrots Ferry Road-broadway Street Between ...,"Widen Road, Upgrade Right Turn Lane, Install L...",6,NON-MPO,2020-01-01
8678,BRLS,5244(032),Patterson,2015-07-08,88530.00,0.00,100000.00,Authorized,NaN,10,STANCOG,Rogers Road Over Delta-mendota Canal (bridge 3...,Bridge Rehabilitation,1,SCAG,2015-01-01
18702,ER,32L0(128),Humboldt County,2021-04-02,0.00,0.00,0.00,Authorized,5904,1,NaN,Bell Spring Road Pm 4.96,Restoration Of Failing Roadway.,4,NON-MPO,2021-01-01
2335,BPMPL,5135(039),Concord,2018-02-12,0.00,0.00,18089.50,Authorized,5135,4,VAR110045,Cty Of Concord Bridge Prev. Maintenance,Bridge Preventive Maintenance,5,MTC,2018-01-01
16456,CML,5223(020),Corcoran,2020-04-02,49479.00,0.00,55890.00,Authorized,5223,6,GP BIKE,Dairy Avenue Between Ottawa Ave And Orange Ave...,30 Ada Ramps Construct Sidewalk. Establish Cla...,1,KCAG,2020-01-01


## Full Function for Prefix

In [5]:
    @interact
    
    def dla_get_prefix(place=df.agency.sort_values().unique().tolist()):
        
        agencies = df[df.agency==place]
    
        prefix_count_n = agencies >> count(_.prefix)
    
        display(Markdown(f"**Summary Statistics for {place}**"))
        display(Markdown(f"The number of obligations {place} has is {len(agencies)}"))
        
        display(Markdown(f"The number of prefix codes {place} uses is {len(prefix_count_n)}"))
        
        
        
        # for the table- using one as some agencies only have one entry
        pd.set_option("display.max_columns", None)
        display(df[df.agency == place][['fed_requested','ac_requested','total_requested']].describe())
        
        display(Markdown(f"**Top Project Types in {place}**"))
        display((df[df.agency == place] >> count(_.type_of_work) >> arrange(-_.n)).head(5)) 
        # graphs 
    
         

    
        ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', y='Number of Obligations', fill="Program Type")
        )
        return ax1

        

interactive(children=(Dropdown(description='place', options=('Access Services', 'Agoura Hills', 'Ala-Con Costa…

## Function for results by Prefix

In [6]:
@interact
def prefix_all_agencies_4(prefix=df.prefix.sort_values().unique().tolist()):
    
    
    # graphs 
    prefixes = df[df.prefix== prefix]
    
    prefix_count_num = (prefixes >> count(_.agency) >> arrange(-_.n))
    
    prefix_count = (prefixes >> count(_.agency) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1
              

interactive(children=(Dropdown(description='prefix', options=('6140000', 'ACNHPI', 'ACST-ER', 'ACSTP', 'ASCTP'…

## Function for Type of Work

In [7]:
top50 = (df >> count(_.type_of_work) >> arrange(-_.n)).head(50)

In [8]:
@interact
def findtypeofwork(work=top50.type_of_work.sort_values().unique().tolist()):
    
    display(Markdown(f"**Number of Obligations the top Agencies have in {work}**"))
    # graphs 
    worktypes = df[df.type_of_work== work]
    
    work_count = (worktypes >> count(_.agency) >> arrange(-_.n)).head(20)
    display(work_count)

    ax1 = (work_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Agencies using Type of Work', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1

interactive(children=(Dropdown(description='work', options=('Ac Overlay', 'Ac Overlay (tc)', 'Asphalt Concrete…

## Function to Filter Data

In [9]:
df_query = df[['prefix', 'agency', 'project_no','locode','dist','fed_requested', 'ac_requested', 'total_requested',
              'type_of_work','project_location', 'seq', 'mpo','prepared_y']]

In [10]:
#adding year
@interact
def get_query2(agency=df_query.agency.unique().tolist(),
             prefix=df_query.prefix.unique().tolist(),
             year=df_query.prepared_y.unique().tolist()):
   
    filtering = df_query[(df_query.agency==agency)&(df_query.prefix==prefix)&(df_query.prepared_y==year)]
    
    
    display(Markdown(f"**The number of obligations {agency} has in {prefix} during the year {year} is {len(filtering)}**"))
    
    display(filtering.head())
    
    

interactive(children=(Dropdown(description='agency', options=('Humboldt County', 'Mendocino County', 'Sacramen…